In [1]:
! pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 202.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Image-preprocessing

In [2]:
import cv2
import numpy as np
import requests
import json

# Function load image from disk
def load_image(image_path):
    return cv2.imread(image_path)

# Function to preprocess image as per model requirements
# More details at: https://github.com/onnx/models/tree/main/validated/vision/classification/resnet
def preprocess_image(img):
    # Resize image to minimum size of 256x256 while maintaining aspect ratio
    min_size = min(img.shape[:2])
    scale_factor = 256 / min_size
    new_size = (int(img.shape[1] * scale_factor), int(img.shape[0] * scale_factor))
    img_resized = cv2.resize(img, new_size)

    # Crop 224x224 from the center
    center_x = new_size[0] // 2
    center_y = new_size[1] // 2
    half_crop = 112
    img_cropped = img_resized[center_y - half_crop:center_y + half_crop, center_x - half_crop:center_x + half_crop]

    # Normalize pixel values
    mean = np.array([0.485, 0.456, 0.406]) * 255
    std = np.array([0.229, 0.224, 0.225]) * 255
    img_normalized = (img_cropped - mean) / std

    # Transpose image from HWC to CHW layout
    img_transposed = np.transpose(img_normalized, (2, 0, 1))

    return img_transposed

# Function to convert image data to flat array
def image_to_flat_array(image_data):
    return image_data.flatten().tolist()

# Function to convert image data to JSON format
def image_to_json(image_data):
    return json.dumps({"inputs": [{"name": "data", "shape": [1, 3, 224, 224], "datatype": "FP32", "data": image_data}]})

## Testing locally yields results for the deployed model on OpenShift AI

In [8]:
# Path to the image file
image_path = 'bee.jpg'

# Load image
image = load_image(image_path)

# Preprocess image
image_processed = preprocess_image(image)

# Convert image to flat array and JSON format
image_flat = image_to_flat_array(image_processed)
image_json = image_to_json(image_flat)

# Send request to OpenVINO server
url = 'https://testing-mlops.apps.cluster-5djnl.dynamic.redhatworkshops.io/v2/models/testing/infer'
headers = {'Content-Type': 'application/json'}
try:
    response = requests.post(url, data=image_json, headers=headers)
    if response.status_code == 200:
        # Parse response
        results = json.loads(response.text)

        # Get the top-5 class labels
        with open('imagenet_classes.txt', 'r') as f:
            class_labels = f.read().splitlines()

        # Get the top-1 prediction
        predictions = np.array(response.json()['outputs'][0]['data'])
        top_prediction_idx = np.argmax(predictions)
        top_prediction_label = class_labels[top_prediction_idx]

        print("Top prediction:", top_prediction_label)
except Exception as e:
    print("Error:", e)

Top prediction: n02206856 bee


## Hosting a Flask application on port 5000.
## Sending a POST request for prediction from the proxy server.

In [10]:
! pip install flask
from flask import Flask, request, render_template
import requests
import json
import numpy as np

app = Flask(__name__)

# Function to preprocess image (replace this with your actual preprocessing function)
def preprocess_image(image):
    # Your preprocessing code here
    return image_processed

# Function to load class labels
def load_class_labels():
    with open('imagenet_classes.txt', 'r') as f:
        class_labels = f.read().splitlines()
    return class_labels

# Function to perform inference
def perform_inference(image):
    # Preprocess image
    image_processed = preprocess_image(image)
    
    # Convert image to flat array and JSON format
    image_flat = image_to_flat_array(image_processed)
    image_json = image_to_json(image_flat)

    # Send request to OpenVINO server
    url = 'https://testing-mlops.apps.cluster-5djnl.dynamic.redhatworkshops.io/v2/models/testing/infer'
    headers = {'Content-Type': 'application/json'}
    try:
        response = requests.post(url, data=image_json, headers=headers)
        if response.status_code == 200:
            # Parse response
            results = json.loads(response.text)

            # Get class labels
            class_labels = load_class_labels()

            # Get the top-1 prediction
            predictions = np.array(response.json()['outputs'][0]['data'])
            top_prediction_idx = np.argmax(predictions)
            top_prediction_label = class_labels[top_prediction_idx]

            return top_prediction_label
    except Exception as e:
        return "Error: {}".format(e)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return render_template('index.html', message='No file part')
        file = request.files['file']
        # If user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            return render_template('index.html', message='No selected file')
        if file:
            # Perform inference
            result = perform_inference(file)
            return render_template('result.html', prediction=result)
    return render_template('index.html')


if __name__ == "__main__":
  app.run(debug=False, host="0.0.0.0", port=5000)



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.132.2.11:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2024 16:08:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 16:08:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 16:08:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 16:08:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 16:09:35] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 16:24:47] "POST / HTTP/1.1" 200 -
